In [2]:
import sys
sys.path.append('./lib')  # for custom modules if any

import cv2
import numpy as np
import streamlit as st
from PIL import Image
import torch
import kornia as K
import kornia.feature as KF

# Import custom SIFT pipeline
from sift_functions import generate_image_pyramid, calculate_dog, SIFT_feature_detection
from helper_functions import quick_resize
from image_stitcher import convert_keypoints_to_cv2, get_descriptors, match_keypoints

# Load LoFTR model
@st.cache_resource
def load_loftr():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = KF.LoFTR(pretrained='outdoor').to(device).eval()
    if device.type == 'cuda':
        model = model.half()
    return model, device

loftr_model, device = load_loftr()
use_half = device.type == 'cuda'

# LoFTR stitching logic
@torch.inference_mode()
def stitch_with_loftr(img1_bgr, img2_bgr):
    img1_gray = cv2.cvtColor(img1_bgr, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2_bgr, cv2.COLOR_BGR2GRAY)

    tensor1 = K.image_to_tensor(img1_gray, False).to(device)
    tensor2 = K.image_to_tensor(img2_gray, False).to(device)

    if use_half:
        tensor1 = tensor1.half() / 255.0
        tensor2 = tensor2.half() / 255.0
    else:
        tensor1 = tensor1.float() / 255.0
        tensor2 = tensor2.float() / 255.0

    batch = {"image0": tensor1, "image1": tensor2}
    output = loftr_model(batch)

    mkpts0 = output['keypoints0'].cpu().numpy()
    mkpts1 = output['keypoints1'].cpu().numpy()

    if len(mkpts0) < 4:
        return None, "❌ Not enough matches found."

    H, _ = cv2.findHomography(mkpts1, mkpts0, cv2.RANSAC, 5.0)
    if H is None:
        return None, "❌ Homography estimation failed."

    return warp_images(img1_bgr, img2_bgr, H), None

# Common warping function

def warp_images(img1, img2, H):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    corners2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
    transformed_corners = cv2.perspectiveTransform(corners2, H)
    all_corners = np.concatenate((np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2), transformed_corners), axis=0)

    [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

    translation = [-xmin, -ymin]
    H_translation = np.array([[1, 0, translation[0]], [0, 1, translation[1]], [0, 0, 1]])

    panorama = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
    panorama[translation[1]:h1 + translation[1], translation[0]:w1 + translation[0]] = img1

    return panorama

# SIFT-based stitching logic
def stitch_with_sift(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    octaves1 = generate_image_pyramid(gray1)
    octaves2 = generate_image_pyramid(gray2)
    dog1 = calculate_dog(octaves1)
    dog2 = calculate_dog(octaves2)

    kp1 = SIFT_feature_detection(dog1, gray1)
    kp2 = SIFT_feature_detection(dog2, gray2)

    cv2_kp1 = convert_keypoints_to_cv2(kp1)
    cv2_kp2 = convert_keypoints_to_cv2(kp2)
    kp1, desc1 = get_descriptors(gray1, cv2_kp1)
    kp2, desc2 = get_descriptors(gray2, cv2_kp2)

    matches = match_keypoints(desc1, desc2)
    if len(matches) < 4:
        return None, "❌ Not enough matches."

    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    if H is None:
        return None, "❌ Homography computation failed."

    return warp_images(img1, img2, H), None

# ------------------------ Streamlit Frontend ------------------------ #
st.set_page_config(layout="wide")
st.title("📸 Image Stitching: SIFT vs LoFTR")
st.markdown("Upload LEFT and RIGHT images, and select your preferred stitching method.")

method = st.radio("Choose a method:", ["SIFT", "LoFTR", "Both"], horizontal=True)

left_file = st.file_uploader("Upload LEFT image", type=["jpg", "jpeg", "png"], key="left")
right_file = st.file_uploader("Upload RIGHT image", type=["jpg", "jpeg", "png"], key="right")

def load_image(file):
    img = Image.open(file).convert("RGB")
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

if left_file and right_file:
    img1 = load_image(left_file)
    img2 = load_image(right_file)
    st.success("✅ Images loaded successfully!")

    if st.button("🧵 Stitch Images"):
        if method == "SIFT":
            with st.spinner("Stitching using SIFT..."):
                result, error = stitch_with_sift(img1, img2)

        elif method == "LoFTR":
            with st.spinner("Stitching using LoFTR (deep learning)..."):
                result, error = stitch_with_loftr(img1, img2)

        elif method == "Both":
            col1, col2 = st.columns(2)
            with st.spinner("Running both methods..."):
                res_sift, err1 = stitch_with_sift(img1, img2)
                res_loftr, err2 = stitch_with_loftr(img1, img2)

            if res_sift is not None:
                col1.image(cv2.cvtColor(res_sift, cv2.COLOR_BGR2RGB), caption="SIFT Panorama", use_column_width=True)
            else:
                col1.error(err1)

            if res_loftr is not None:
                col2.image(cv2.cvtColor(res_loftr, cv2.COLOR_BGR2RGB), caption="LoFTR Panorama", use_column_width=True)
            else:
                col2.error(err2)
            st.stop()

        if result is not None:
            st.image(cv2.cvtColor(result, cv2.COLOR_BGR2RGB), caption=f"🧵 Final Panorama ({method})", use_column_width=True)
        else:
            st.error(error)


2025-04-09 01:48:37.501 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-09 01:48:37.677 Session state does not function when running a script without `streamlit run`
